# Segmenting and Clustering Neighborhoods in Toronto Part 2

## First of all, it is neccesary run the part 1 in order to obtain the wikipedia data 

### Import libraries

In [2]:
import pandas as pd
import numpy as np
import requests

### Scrapping the wikipedia data

In [3]:
wiki_url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
wiki_page = requests.get(wiki_url)
wiki_data = pd.read_html(wiki_page.text)
wiki_data

[    Postal Code           Borough  \
 0           M1A      Not assigned   
 1           M2A      Not assigned   
 2           M3A        North York   
 3           M4A        North York   
 4           M5A  Downtown Toronto   
 ..          ...               ...   
 175         M5Z      Not assigned   
 176         M6Z      Not assigned   
 177         M7Z      Not assigned   
 178         M8Z         Etobicoke   
 179         M9Z      Not assigned   
 
                                          Neighbourhood  
 0                                         Not assigned  
 1                                         Not assigned  
 2                                            Parkwoods  
 3                                     Victoria Village  
 4                            Regent Park, Harbourfront  
 ..                                                 ...  
 175                                       Not assigned  
 176                                       Not assigned  
 177                

In [4]:
len(wiki_data)

3

In [5]:
type(wiki_data)

list

### Obtain dataframe with three columns

In [6]:
#only it is neccesary the first table so we can drop the other tables

wiki_df = wiki_data[0]
wiki_df.head()
wiki_df

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
...,...,...,...
175,M5Z,Not assigned,Not assigned
176,M6Z,Not assigned,Not assigned
177,M7Z,Not assigned,Not assigned
178,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."


In [7]:
list(wiki_df.columns.values)

['Postal Code', 'Borough', 'Neighbourhood']

### Drop the Borough and Neighbourhood which are not assigned

In [8]:
df = wiki_df[wiki_df['Neighbourhood'] != 'Not assigned']
df.head()

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [9]:
df.Neighbourhood.str.count("Not assigned").sum()

0

### Group by postal codes, reset the index values and drop the old index

In [10]:
# Group by Postal Code

df = df.sort_values(['Postal Code'])
df



,Postal Code,Borough,Neighbourhood
9,M1B,Scarborough,"Malvern, Rouge"
18,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
27,M1E,Scarborough,"Guildwood, Morningside, West Hill"
36,M1G,Scarborough,Woburn
45,M1H,Scarborough,Cedarbrae
...,...,...,...
98,M9N,York,Weston
107,M9P,Etobicoke,Westmount
116,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ..."
143,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest..."


In [11]:
#Reset the index values and drop the old index
df.reset_index(inplace = True)
df.drop(['index'], axis = 'columns', inplace = True)
df

,Postal Code,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
...,...,...,...
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ..."
101,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest..."


### Obtain the shape of dataframe

In [12]:
df.shape

(103, 3)

## Part 2

### Install geocoder and obtain the coords

In [22]:
!pip install geocoder

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 98 kB 1.8 MB/s eta 0:00:011


In [ ]:
import geocoder

# initialize your variable to None
#lat_lng_coords = None

#postal_code = 'M3A'

# loop until you get the coordinates
#while(lat_lng_coords is None):
  #g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
  #lat_lng_coords = g.latlng

#latitude = lat_lng_coords[0]
#longitude = lat_lng_coords[1]


### After 30' the code does not obtain the coordinates, so I decide obtain the coordinates with the alternative way offer in the course, importing csv file from the URL

In [13]:
geodata = pd.read_csv("https://cocl.us/Geospatial_data")
geodata

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


### Check the shapes and types of the two dataframes 

In [14]:
print("The wiki data shape is ", df.shape)
print("The geodata shape is ", geodata.shape)

The wiki data shape is  (103, 3)
The geodata shape is  (103, 3)


In [18]:
print("The wiki data types are \n", df.dtypes, "\n")

print("The geodata types are \n", geodata.dtypes)

The wiki data types are 
 Postal Code      object
Borough          object
Neighbourhood    object
dtype: object 

The geodata types are 
 Postal Code     object
Latitude       float64
Longitude      float64
dtype: object


### Join the two data in one dataframe using pandas

In [20]:
merge_data = df.join(geodata.set_index('Postal Code'), on = 'Postal Code', how = 'inner')
merge_data

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
...,...,...,...,...,...
98,M9N,York,Weston,43.706876,-79.518188
99,M9P,Etobicoke,Westmount,43.696319,-79.532242
100,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ...",43.688905,-79.554724
101,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest...",43.739416,-79.588437


In [21]:
merge_data.shape

(103, 5)